In [34]:
from jupyterirtools import sso
import os
import time
import importlib

sso.login("Jupyter-IR-AdministratorAccess", os.environ['MANAGEMENT_ACCOUNT'])

If the windows doesn't automatically open, click on this https://device.sso.us-east-1.amazonaws.com/?user_code=KXVH-JCCH to activate the session


<IPython.core.display.Javascript object>

In [35]:
!echo $AWS_PROFILE 

Jupyter-IR-AdministratorAccess-613375713311


# Organization Configuration
This notebook will configure the organization to have the resources needed to run the Jupyter Incident Response runbooks across the organization. This **must be run in the organizational management account.**.

Thise notebook is idempotent - you can run it multiple times and it will not create any duplicate resources. It used cloudformation to manage the resources, and redeploying the stack will only change settings that require changes.

The Athena tables can be set up in a delagate account so no resources will be deployed into the management account.



## Accounts:
* **Management Account**: The root account that owns the AWS organization. Typically resources are note deployed to the root account, but some configurations must be done in this account. For example, the cloudtrail organizational trail is created in the management account.
* **Logging**: The account that holds the log files. Typically this account contains the buckets for the CloudTrail and other logs, and Athena tables used to query them.
* **CloudFormation StackSet Delegated Account**: An account designated by the management account to own stacksets.

## Parameters
The following cell holds the parameters for the environment

* **external_cloud_trail**: Set this to true if organizational cloudtrail logs have already been configured.
* **athena_database**: The name of the athena database that will be used to query logs in S3.
* **athena_cloudtrail_table**: The name of the athena table that will be used to query CloutTrail logs in S3.
* **flow_logs_bucket**: The bucket that holds the organizational flow logs. Leave empty and one will be created automatically.
* **enable_flow_logs_automatically**: Automatically enable flow logs in all VPCs in the organization. See [blog](https://aws.amazon.com/blogs/mt/how-to-enable-vpc-flow-logs-automatically-using-aws-config-rules/)
* **opt_in_out**: Use this setting to use opt-in for regions or opt-out. set to **in** for opt in, and **out** for opt out.
* **region_list**: Along with the opt_in_out setting, this will list the regions to opt-in or opt-out.
* **aggregator_id**: Set this to the name of the organizational aggregator, if one is already available. If left empty, a new organizational aggregator will be created.

In [47]:
# Save Settings


external_cloud_trail = True # True | False. Set this to True if the organization cloudtrail is managed externally.
athena_database = "jupyter-incident-response"  # <--- To customize, replace the value with the athena database name
athena_cloudtrail_table = "cloudtrail_logs"  # <--- To customize, replace the value with the athena table name for cloudtrail logs
flow_logs_bucket = "aws-controltower-logs-279450423037-us-east-1"  # <--- Leave blank and a bucket will be created automatically
flowlog_output_format = "parquet" # plain-text | parquet  parqet is the best option
cloudtrail_bucket = "aws-controltower-logs-279450423037-us-east-1"  # <--- Leave blank and a bucket will be created automatically
enable_flow_logs_automatically = True # True | False
opt_in_out = "in" # in | out
region_list = "us-east-1,us-west-2,us-east-2" # comma separated
aggregator_id = ""  # <--- Leave blank and an aggregator will be created automatically
config_bucket = "aws-controltower-logs-279450423037-us-east-1"

## CloudFormation
This notebook uses cloudformation to deploy the cloudformation stacks and stack sets used to configure the organization. 

The templates have feature toggles to turn on an off the creation of the configuration. If the resources have already been created outside of this notebook, the resource creation will be skipped.
* **sso-jupyter-server.yaml**: Creates a SageMake notebook and loads this repository.
* **sso-environment.yaml**: Configures the permission sets and groups in IAM Identity Center 
* **logging-account.yaml**: 
  * If necessary. creates the S3 buckets for organization-wide logging
  * Creates Athena tables to query logs
* **account-level-resources.yaml**:
  * Note: Deployed as a stackset from the **Delegated Stack Stack** account to every account under the root.
  * **account-configurations.yaml**: Creates global resources in each member account.
  * **region-account-configutations.yaml**: Creates a stackset to deployed to each region in the account
    * Configures Flow Log Auto Remediation
* **managage-account.yaml**: If necessary, configures organizational CloudTrail logs.
* **aws-config-aggregator.yaml**: Configures the AWS Config settings across the org. This will only create resources if required.

# Organizational Onboarding

The notebook will validate and update the organization to configure the logging and configuration of the organization to the other runbooks have access to the data and services needed to execute the runbooks.

This runbook is designed to be able to be run multiple times, so it will not create additional resources if the configurate has already been found.

The following services will be configured.
* **CloudTrail**: An organization trail will be configured to store the CloudTrail logs in a bucket in the management account. The first trail in each account is included free of charge. If an organzation trail already exists, a new one will not be created.
* **VPC Flow Logs**: Any VPCs with the tag 'runbooks-enable-flowlogs' set to 'true' will have VPC flow logs enabled into a central bucket.
* **Athena**: Athena will be configured to query the following service logs:
  * CloudTrail
  * VPC Flow Logs
  * AWS Config
  * ***Future*** Application Load Balancer
  * ***Future*** Elastic Load Balancer
  * ***Future*** CloudFront
  * ***Future*** Amazon EMR
  * ***Future*** Global Accelerator
  * ***Future*** GuardDuty
  * ***Future*** Network Firewall
  * ***Future*** Network Load Balancer
  * ***Future*** Route 53
  * ***Future*** Amazon SES
  * ***Future*** AWS WAF
  

## Load Settings

The information on how to connect to data in the AWS environment is managed through SSM Parameter Store.

The athena database and tablenames can be provided below. The runbook will manage the Athena tables so they are configured with the columns and partitioning expected by the notebooks.

In [48]:
# Load Settings
import boto3
ssm_client = boto3.client('ssm')

put_response = ssm_client.put_parameter(Name='Jupyter-Athena-Database', Value=athena_database, Type="String", Overwrite=True)
sso_portal_url_response = ssm_client.put_parameter(Name='Jupyter-Athena-Cloudtrail', Value=athena_cloudtrail_table, Type="String", Overwrite=True)

get_management_account_response = ssm_client.get_parameter(Name='Jupyter-Management-Account')
get_cfn_delegate_account_response = ssm_client.get_parameter(Name='Jupyter-CfnDelegateAccount')
get_logging_account_response = ssm_client.get_parameter(Name='Jupyter-LoggingAccount')

management_account = get_management_account_response['Parameter']['Value']
cfn_delegate_account = get_cfn_delegate_account_response['Parameter']['Value']
logging_account = get_logging_account_response['Parameter']['Value']

ec2_client = boto3.client('ec2')

stack_set_region_list = ""

if opt_in_out == "out":
    region_opt_out = region_list.split(',')
    stack_set_regions = []

    region_response = ec2_client.describe_regions()

    for region in region_response['Regions']:
        if not region['RegionName'] in region_opt_out:
            stack_set_regions.append(region['RegionName'])
    stack_set_region_list =','.join(stack_set_regions)
else:
    stack_set_region_list = region_list

management_account

'613375713311'

# Deploy the organizational logging Athena 
The following CloudFormation will deploy the organizational cloudtrail logs athena tables into the logging account.

In [49]:
!echo cloudformation deploy --template-file cfn-templates/logging-account.yaml --stack-name organization-logging --parameter-overrides ExistingConfigBucket=$config_bucket RegionList=$stack_set_region_list FlowLogsFormat=$flowlog_output_format DatabaseName=$athena_database CloudTrailLogsBucket=$cloudtrail_bucket FlowLogsBucket=$flow_logs_bucket OrgId=$org_id CloudTrailTableName=$athena_cloudtrail_table --capabilities CAPABILITY_IAM --profile $profile


cloudformation deploy --template-file cfn-templates/logging-account.yaml --stack-name organization-logging --parameter-overrides ExistingConfigBucket=aws-controltower-logs-279450423037-us-east-1 RegionList=us-east-1,us-west-2,us-east-2 FlowLogsFormat=parquet DatabaseName=jupyter-incident-response CloudTrailLogsBucket=aws-controltower-logs-279450423037-us-east-1 FlowLogsBucket=aws-controltower-logs-279450423037-us-east-1 OrgId=o-zbnt7bbfyh CloudTrailTableName=cloudtrail_logs --capabilities CAPABILITY_IAM --profile Jupyter-IR-AdministratorAccess-279450423037


In [55]:
# Configure Logging Account
org_client = boto3.client('organizations')
desc_client = org_client.describe_organization()

org_id = desc_client['Organization']['Id']

session = sso.get_session("Jupyter-IR-AdministratorAccess", logging_account)

profile = f"Jupyter-IR-AdministratorAccess-{logging_account}"

!aws cloudformation deploy --template-file cfn-templates/logging-account.yaml --stack-name organization-logging --parameter-overrides ExistingConfigBucket=$config_bucket RegionList=$stack_set_region_list FlowLogsFormat=$flowlog_output_format DatabaseName=$athena_database CloudTrailLogsBucket=$cloudtrail_bucket FlowLogsBucket=$flow_logs_bucket OrgId=$org_id CloudTrailTableName=$athena_cloudtrail_table --capabilities CAPABILITY_IAM --profile $profile


Waiting for changeset to be created..
Waiting for stack create/update to complete
Successfully created/updated stack - organization-logging


### Get the account settings

Pull the settings out of the cloudformation stacks.

In [18]:
config_bucket = !aws cloudformation describe-stacks --stack-name organization-logging  --profile $profile --query "Stacks[0].Outputs[?OutputKey=='ConfigBucket'].OutputValue" --output text
config_bucket = config_bucket[0]

config_topic = !aws cloudformation describe-stacks --stack-name organization-logging  --profile $profile --query "Stacks[0].Outputs[?OutputKey=='ConfigTopic'].OutputValue" --output text
config_topic = config_topic[0]


cfn_cloudtrail_bucket = !aws cloudformation describe-stacks --stack-name organization-logging --profile $profile  --query "Stacks[0].Outputs[?OutputKey=='CloudTrailLogsBucket'].OutputValue" --output text
cfn_flowlog_bucket = !aws cloudformation describe-stacks --stack-name organization-logging --profile $profile  --query "Stacks[0].Outputs[?OutputKey=='BucketName'].OutputValue" --output text

cfn_cloudtrail_bucket = cfn_cloudtrail_bucket[0]
cfn_flowlog_bucket = cfn_flowlog_bucket[0]

print(f"config_bucket={config_bucket}\nconfig_topic={config_topic}\ncfn_cloudtrail_bucket={cfn_cloudtrail_bucket}\ncfn_flowlog_bucket={cfn_flowlog_bucket}")


config_bucket=aws-controltower-logs-279450423037-us-east-1
config_topic=arn:aws:sns:us-east-1:279450423037:config-topic-279450423037-organization-logging
cfn_cloudtrail_bucket=aws-controltower-logs-279450423037-us-east-1
cfn_flowlog_bucket=aws-controltower-logs-279450423037-us-east-1


In [30]:

cfn_profile = f"Jupyter-IR-AdministratorAccess-{cfn_delegate_account}"
session = sso.get_session("Jupyter-IR-AdministratorAccess", cfn_delegate_account)

!aws cloudformation deploy --template-file cfn-templates/cfn-bucket.yaml --stack-name cfn-bucket --parameter-overrides OrgId=$org_id --profile $cfn_profile

cfn_bucket = !aws cloudformation describe-stacks --stack-name cfn-bucket --profile $cfn_profile --query "Stacks[0].Outputs[?OutputKey=='CfnBucket'].OutputValue" --output text
cfn_bucket = cfn_bucket[0]



Waiting for changeset to be created..

No changes to deploy. Stack cfn-bucket is up to date


## Validate CloudTrail
The next section will check the CloudTrail logs and validate an organizational trail is configured.

Then the cloudformation stack will be created in the management account to configure the organization trail if one is needed.

## Account and Region Configurations

Any configurations that need to deployed to each account will be handled with this stackset.


In [24]:


org_trail_info = {}
org_trail_arn = ""
found_org_trail = True

cloudtrail_client = boto3.client('cloudtrail', region_name = "us-east-1")

if external_cloud_trail == True:
    found_org_trail = False

    paginator = cloudtrail_client.get_paginator('list_trails')

    response_iterator = paginator.paginate()
    for page in response_iterator:
        for trail in page["Trails"]:
            trail_response = cloudtrail_client.get_trail(Name=trail['TrailARN'])
            if trail_response['Trail']['IsOrganizationTrail'] and trail_response['Trail']['IsMultiRegionTrail']:
                found_org_trail = True
                org_trail_info = trail_response['Trail']
                org_trail_arn = org_trail_info['TrailARN']

    if not found_org_trail:
        raise ("External cloud trail not found, please configure.")

!aws cloudformation deploy --template-file cfn-templates/managage-account.yaml --stack-name management-organization --parameter-overrides LoggingAccount=$logging_account CloudTrailArn=$org_trail_arn TrailBucket=$cfn_cloudtrail_bucket --capabilities CAPABILITY_IAM



Waiting for changeset to be created..
Waiting for stack create/update to complete
Successfully created/updated stack - management-organization


In [25]:
trail_arn = !aws cloudformation describe-stacks --stack-name management-organization --query "Stacks[0].Outputs[?OutputKey=='TrailArn'].OutputValue" --output text

trail_response = cloudtrail_client.get_trail(Name=trail_arn[0])
org_trail_info = trail_response['Trail']

print(f"Organization Trail was found: {org_trail_info['Name']} in region {org_trail_info['HomeRegion']}")

Organization Trail was found: aws-controltower-BaselineCloudTrail in region us-east-1


# Configure member accounts for flow logs and AWS Config
The next section will create a CloudFormation stack that configures the AWS Config and flowlogs in every account and every region.



In [31]:

org_unit_id = org_client.list_roots()['Roots'][0]['Id']
cfn_profile

'Jupyter-IR-AdministratorAccess-613375713311'

In [36]:
import random, string

config_bucket = '' # Disable config recorder 


random_str=''.join(random.choices(string.ascii_uppercase + string.digits, k=8))

!aws s3 sync cfn-templates/ s3://$cfn_bucket/$random_str/ --profile $cfn_profile > /dev/null

profile = f"Jupyter-IR-AdministratorAccess-{cfn_delegate_account}"
template_url = f'https://s3.us-east-1.amazonaws.com/{cfn_bucket}/{random_str}'

!aws cloudformation deploy --stack-name jupyter-ir-config --template-file cfn-templates/organization-ir-config.yaml --parameter-overrides FlowLogsBucket=flow_logs_bucket ConfigTopic=$config_topic ConfigBucket=$config_bucket OrgUnitId=$org_unit_id OutputFormat=$flowlog_output_format  AutomateRemediationTemplateUrl=$template_url RegionList=$stack_set_region_list --profile $profile --capabilities CAPABILITY_NAMED_IAM




Waiting for changeset to be created..
Waiting for stack create/update to complete
Successfully created/updated stack - jupyter-ir-config


In [37]:

profile = f"Jupyter-IR-AdministratorAccess-{logging_account}"
!aws s3 rm --recursive s3://$cfn_bucket/$random_str --profile $profile > /dev/null

delete failed: s3://cfn-bucket-cfnbucket-1mrtfyxtk988y/PF7MGTPE/.ipynb_checkpoints/cfn-bucket-checkpoint.yaml An error occurred (AccessDenied) when calling the DeleteObject operation: Access Denied
delete failed: s3://cfn-bucket-cfnbucket-1mrtfyxtk988y/PF7MGTPE/organization-ir-config.yaml An error occurred (AccessDenied) when calling the DeleteObject operation: Access Denied
delete failed: s3://cfn-bucket-cfnbucket-1mrtfyxtk988y/PF7MGTPE/sso-environment.yaml An error occurred (AccessDenied) when calling the DeleteObject operation: Access Denied
delete failed: s3://cfn-bucket-cfnbucket-1mrtfyxtk988y/PF7MGTPE/account-configurations.yaml An error occurred (AccessDenied) when calling the DeleteObject operation: Access Denied
delete failed: s3://cfn-bucket-cfnbucket-1mrtfyxtk988y/PF7MGTPE/.ipynb_checkpoints/managage-account-checkpoint.yaml An error occurred (AccessDenied) when calling the DeleteObject operation: Access Denied
delete failed: s3://cfn-bucket-cfnbucket-1mrtfyxtk988y/PF7MGTPE/s

## Test CloudTrail access with Athena
The following code uses a helper library to query cloudtrail logs and return them as a dataframe

In [57]:
from jupyterirtools import athena
import importlib
importlib.reload(athena)


accountid = os.environ['MANAGEMENT_ACCOUNT']

sql = f"""
SELECT * 
FROM \"{athena_database}\".\"{athena_cloudtrail_table}\"
WHERE accountId = '{accountid}'
LIMIT 10
"""

display(athena.run_query(sql))


profile_name=Jupyter-IR-ViewOnly-279450423037


,eventversion,useridentity,eventtime,eventsource,eventname,awsregion,sourceipaddress,useragent,errorcode,errormessage,...,eventtype,apiversion,readonly,recipientaccountid,serviceeventdetails,sharedeventid,vpcendpointid,accountid,region,timestamp
0,1.08,"{type=AWSService, principalid=null, arn=null, ...",2022-10-31T05:34:36Z,sts.amazonaws.com,AssumeRole,us-west-2,cloudtrail.amazonaws.com,cloudtrail.amazonaws.com,None,None,...,AwsApiCall,None,true,613375713311,None,d25e482d-b095-448b-a2c3-bacf49aeeec2,None,613375713311,us-west-2,2022/10/31
1,1.08,"{type=AWSService, principalid=null, arn=null, ...",2022-10-31T05:36:19Z,sts.amazonaws.com,AssumeRole,us-west-2,cloudtrail.amazonaws.com,cloudtrail.amazonaws.com,None,None,...,AwsApiCall,None,true,613375713311,None,ed87d95b-8f66-485e-aab9-1f7a6d82b0d9,None,613375713311,us-west-2,2022/10/31
2,1.08,"{type=AWSService, principalid=null, arn=null, ...",2022-10-31T05:36:23Z,sts.amazonaws.com,AssumeRole,us-west-2,cloudtrail.amazonaws.com,cloudtrail.amazonaws.com,None,None,...,AwsApiCall,None,true,613375713311,None,7b5daa65-f3ab-4c1d-9952-1a59984673b5,None,613375713311,us-west-2,2022/10/31
3,1.08,"{type=AWSService, principalid=null, arn=null, ...",2022-10-31T05:37:19Z,sts.amazonaws.com,AssumeRole,us-west-2,cloudtrail.amazonaws.com,cloudtrail.amazonaws.com,None,None,...,AwsApiCall,None,true,613375713311,None,a0ef0691-943f-44a8-a461-2b4f2ca6b8d9,None,613375713311,us-west-2,2022/10/31
4,1.08,"{type=AWSService, principalid=null, arn=null, ...",2022-10-31T05:38:58Z,sts.amazonaws.com,AssumeRole,us-west-2,cloudtrail.amazonaws.com,cloudtrail.amazonaws.com,None,None,...,AwsApiCall,None,true,613375713311,None,f9caccf6-4a75-416d-9907-542bb61566f6,None,613375713311,us-west-2,2022/10/31
5,1.08,"{type=AssumedRole, principalid=AROAY5UAKUAPZOG...",2022-10-29T15:19:30Z,ssm.amazonaws.com,UpdateInstanceInformation,us-east-1,54.152.27.160,aws-sdk-go/1.41.4 (go1.18.3; linux; amd64) ama...,None,None,...,AwsApiCall,None,false,613375713311,None,None,None,613375713311,us-east-1,2022/10/29
6,1.08,"{type=AWSService, principalid=null, arn=null, ...",2022-10-29T15:19:31Z,sts.amazonaws.com,AssumeRole,us-east-1,ec2.amazonaws.com,ec2.amazonaws.com,None,None,...,AwsApiCall,None,true,613375713311,None,215f8b4c-24f0-46a1-ab15-aa9247f079e5,None,613375713311,us-east-1,2022/10/29
7,1.08,"{type=AWSService, principalid=null, arn=null, ...",2022-10-29T15:19:56Z,sts.amazonaws.com,AssumeRole,us-east-1,codepipeline.amazonaws.com,codepipeline.amazonaws.com,None,None,...,AwsApiCall,None,true,613375713311,None,c21c6cfe-bdfd-4f33-8f22-6269ac3d41c7,None,613375713311,us-east-1,2022/10/29
8,1.08,"{type=AWSService, principalid=null, arn=null, ...",2022-10-29T15:21:56Z,sts.amazonaws.com,AssumeRole,us-east-1,codepipeline.amazonaws.com,codepipeline.amazonaws.com,None,None,...,AwsApiCall,None,true,613375713311,None,c3fb6b3e-9aec-4112-9c82-2d5f10ba25dc,None,613375713311,us-east-1,2022/10/29
9,1.08,"{type=AssumedRole, principalid=AROAY5UAKUAPWJU...",2022-10-29T15:22:35Z,datasync.amazonaws.com,ListLocations,us-east-1,config.amazonaws.com,config.amazonaws.com,None,None,...,AwsApiCall,None,true,613375713311,None,None,None,613375713311,us-east-1,2022/10/29


## Flow Logs
Enable athena to query organizational flow logs.

This needs to be deployed in two steps. 
1. Deploy CFN The central bucket and athena query in the management account
1. The stacksets for all regions/accounts in organization.

In [58]:


display(athena.os.environ['MANAGEMENT_ACCOUNT'])


'613375713311'

In [59]:
accounts = []

paginator = org_client.get_paginator('list_accounts')

page_iterator = paginator.paginate()
for page in page_iterator:
    for account in page['Accounts']:
        accounts.append(account['Id'])

# Delete Flow Logs - delete this in the future or move to another notebook
if False: # Don't do this by default, it exists to delete all the flow logs in an org if the deployment needs to be redone.
    for account in accounts:
        profile = f"Jupyter-IR-AdministratorAccess-{account}"
        !aws s3 rm --recursive s3://$cfn_flowlog_bucket/AWSLogs/$account --profile Jupyter-IR-AdministratorAccess-251344881676
        for region_name in stack_set_region_list.split(','):
            session = sso.get_session("Jupyter-IR-AdministratorAccess", account, region_name)
            client = session.client('ec2')
            response = client.describe_flow_logs()

            flow_log_ids = []
            for flow_log in response['FlowLogs']:
                flow_log_id = flow_log['FlowLogId']
                flow_log_ids.append(flow_log_id)

            if len(flow_log_ids) > 0:
                delete_response = client.delete_flow_logs(FlowLogIds=flow_log_ids)
                print(f"deleted: {flow_log_ids} in region {region_name} in account {account}")

In [60]:

for account in accounts:
    profile = f"Jupyter-IR-AdministratorAccess-{account}"
    for region_name in stack_set_region_list.split(','):
        session = sso.get_session("Jupyter-IR-AdministratorAccess", account, region_name)
        client = session.client('ec2')
        response = client.describe_flow_logs()

        flow_log_ids = []
        for flow_log in response['FlowLogs']:
            flow_log_id = flow_log['FlowLogId']
            options = "cloudwatch"
            if "DestinationOptions" in flow_log:
                options = flow_log['DestinationOptions']['FileFormat']
            print(f"FlowLogId: {flow_log_id}:{options} in region {region_name} in account {account}")

FlowLogId: fl-0c3b874683ab03100:cloudwatch in region us-east-1 in account 811741334843
FlowLogId: fl-024bb0657cee27a12:cloudwatch in region us-east-2 in account 811741334843
FlowLogId: fl-0d7f08976cde7a262:cloudwatch in region us-east-1 in account 963668006045
FlowLogId: fl-019c7e0a00187b2e5:cloudwatch in region us-east-2 in account 963668006045
FlowLogId: fl-0ae554e398c584797:cloudwatch in region us-east-1 in account 017990242238
FlowLogId: fl-07bc2aaf579e3b613:cloudwatch in region us-east-2 in account 017990242238
FlowLogId: fl-019c7a4b3c7e8cf97:cloudwatch in region us-east-1 in account 329925206794
FlowLogId: fl-06ffe76c39c0ff12b:cloudwatch in region us-east-2 in account 329925206794


## AWS Config

It is a best practice to enable AWS config across the organization to collect data on the current state of the resoruces deployed in the organization, establish detective controls for compliance, and make resources available through a query interface.

### Check delivery channels

Verify a delivery channel exists for the every account and every region im the organizational.


In [61]:

accounts = []

paginator = org_client.get_paginator('list_accounts')
page_iterator = paginator.paginate()
for page in page_iterator:        
    for acct in page['Accounts']:
        accounts.append(acct['Id'])

for account in accounts:
    profile = f"Jupyter-IR-AdministratorAccess-{account}"
    for region_name in stack_set_region_list.split(','):
        
        session = sso.get_session("Jupyter-IR-AdministratorAccess", account, region_name)
        config_client = session.client('config')
        delivery_channels_result = config_client.describe_delivery_channels()
        delivery_channel_found = False
        for delivery_channel in delivery_channels_result['DeliveryChannels']:
            delivery_channel_found = True
        
        if delivery_channel_found:
            print(f"Delivery Channel found in account {account} in region {region_name}")
        else:
            print(f"Delivery Channel NOT found in account {account} in region {region_name}")

Delivery Channel found in account 811741334843 in region us-east-1
Delivery Channel found in account 811741334843 in region us-west-2
Delivery Channel found in account 811741334843 in region us-east-2
Delivery Channel found in account 963668006045 in region us-east-1
Delivery Channel found in account 963668006045 in region us-west-2
Delivery Channel found in account 963668006045 in region us-east-2
Delivery Channel found in account 613375713311 in region us-east-1
Delivery Channel found in account 613375713311 in region us-west-2
Delivery Channel found in account 613375713311 in region us-east-2
Delivery Channel found in account 569230450028 in region us-east-1
Delivery Channel found in account 569230450028 in region us-west-2
Delivery Channel found in account 569230450028 in region us-east-2
Delivery Channel found in account 279450423037 in region us-east-1
Delivery Channel found in account 279450423037 in region us-west-2
Delivery Channel found in account 279450423037 in region us-ea

## Set up the multi-account multi-region aggregator

An aggregator is an AWS Config resource type that collects AWS Config configuration and compliance data from the following:

* Multiple accounts and multiple regions.
* Single account and multiple regions.
* An organization in AWS Organizations and all the accounts in that organization which have AWS Config enabled.
* Use an aggregator to view the resource configuration and compliance data recorded in AWS Config.

In [62]:
profile = f"Jupyter-IR-AdministratorAccess-{management_account}"
!aws cloudformation deploy --stack-name aws-config-aggregator --template-file cfn-templates/aws-config-aggregator.yaml --parameter-overrides AggregatorId=$aggregator_id RegionList=$stack_set_region_list --profile $profile --capabilities CAPABILITY_IAM

print('It can take 10 minutes for the aggregator to be ready to field a query')


Waiting for changeset to be created..
Waiting for stack create/update to complete
Successfully created/updated stack - aws-config-aggregator
It can take 10 minutes for the aggregator to be ready to field a query


## Test AWS Config Aggregator Query
The following query will display the first 100 the running EC2 instances in the organization. Remember, it may take a few minutes for the aggregator to completely deployed across the organization.

The **jupyter_awsconfig** is a helper library to simplify calling aws config queries and returning a pandas dataframe. 

In [65]:
import json
from jupyterirtools import awsconfig

import importlib
importlib.reload(awsconfig)

query_expression = f"""
SELECT
  accountId,
  awsRegion,
  resourceId,
  configuration.instanceType,
  availabilityZone,
  configuration.state.name
WHERE
  resourceType = 'AWS::EC2::Instance'
"""

df = awsconfig.run_query(query_expression, 100)
display(df)

,accountId,awsRegion,resourceId,configuration.instanceType,availabilityZone,configuration.state.name
0,613375713311,us-east-1,i-03f0d2cece4bcba61,t2.micro,us-east-1b,running
1,963668006045,us-east-2,i-03fd51ebf05d8743f,t2.micro,us-east-2b,stopped
